In [4]:
from flask import Flask, request, render_template
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
import os

# Flask app setup
app = Flask(__name__)

# Load historical logs
historical_logs = pd.read_csv('data/historical_logs.csv')  # Must have columns: 'text', 'label'

# Set your OpenAI API key (you can use environment variable too)
OPENAI_API_KEY = "sk-UCuBbs9rn2Nx_NQzMgBdqw"

# Initialize Langchain's OpenAI wrapper
chat_model = ChatOpenAI(model="gpt-4", api_key=OPENAI_API_KEY)

# Function to classify text using Langchain + OpenAI
def classify_text_with_langchain(text):
    prompt = f"""
    You are a content moderation assistant. Classify the following text into one of these categories: 
    ["abusive", "spam", "hate speech", "harmless"]. 
    Only return the category.

    Text: "{text}"
    """

    # Use Langchain to send prompt to OpenAI
    response = chat_model.invoke([HumanMessage(content=prompt)])
    category = response.content.strip().lower()

    # Validate category (optional safety check)
    valid_categories = ["abusive", "spam", "hate speech", "harmless"]
    if category not in valid_categories:
        category = "unknown"
    
    return category

# Find similar logs from historical data
def find_similar_logs(text):
    short_text = text[:30].lower()
    similar_logs = historical_logs[historical_logs['text'].str.contains(short_text, case=False, na=False)]
    return similar_logs.to_dict(orient='records')

@app.route('/', methods=['GET', 'POST'])
def index():
    result = None

    if request.method == 'POST':
        user_text = request.form['user_text']

        # Classify text using Langchain (OpenAI)
        category = classify_text_with_langchain(user_text)

        # Find similar historical logs
        similar_logs = find_similar_logs(user_text)

        result = {
            "text": user_text,
            "category": category,
            "historical_matches": similar_logs
        }

    return render_template('index.html', result=result)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

c:\Users\KIIT\Anaconda\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
